In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import skimage
from skimage.io import imshow, imread, imsave
import numpy as np
from glob import glob
import os
import cv2
from matplotlib import pyplot as plt
from PIL import Image
import tensorflow as tf
from tensorflow import keras as K

In [ ]:
#Setting variables to hold the input and output directory paths
in_dir = '/content/drive/My Drive/Original'
out_dir = '/content/drive/My Drive/Otsu_No_Blur'

In [ ]:
#Read the input image file names with paths into a list
in_files = in_dir + '/*.png' #all files with .png extension will be selected
img_names = glob(in_files) #glob returns a list of file names with paths that match the in_files pattern
#print(img_names) #prints the list of files names

In [ ]:
#Applying Otsu's method without blur
for fn in img_names: #fn = file name
    print('processing %s...' % fn) #print a message to show which file is being processed

    # read the input image as gray (0 specifies that the image should be loaded in grayscale)
    im_gray = cv2.imread(fn, 0)


    # threshold it with your saved threshold
                                 #0 is input as the threshold but Otsu's method automatically determines an optimal threshold
                                      # 255 is the maximum value assigned to pixels above the threshold
                                          #binary threshold is inverted (foreground becomes black, background becomes white)
                                                                   #apply Otsu's threshold
                                                                                  #the function returns two values, we only want the second one ie the threshold imagae
    im_bw = cv2.threshold(im_gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

    # write the result to disk in the previously created output directory
    name = os.path.basename(fn) #extract the basename of the file
    outfile = out_dir + '/' + name #creates output file path by combining output directory with basename
    cv2.imwrite(outfile, im_bw) #image is written(saved) to the file path set above

processing /content/drive/My Drive/Original/002f20ad-2ace-499c-9335-c9080bc3e6b5.png...
processing /content/drive/My Drive/Original/003a89b0-a095-417a-8dd6-f408339bbc68.png...
processing /content/drive/My Drive/Original/003d1cdc-0aec-430b-90e0-f4d82135b1ac.png...
processing /content/drive/My Drive/Original/005e60b6-77b8-458c-b57c-bfe0c7e7df78.png...
processing /content/drive/My Drive/Original/00a02700-2ea2-4590-9e15-ffc9160fd3de.png...
processing /content/drive/My Drive/Original/00c8364b-8c85-4502-bcfe-64736fe76815.png...
processing /content/drive/My Drive/Original/00d04a90-80e5-4bce-9511-1b64eabb7a47.png...
processing /content/drive/My Drive/Original/0154dd8e-72f8-4d78-a4dd-93e139577bd1.png...
processing /content/drive/My Drive/Original/01b38a56-13eb-447b-89ee-fc21cb40dcc1.png...
processing /content/drive/My Drive/Original/01b99da8-b66b-464b-a7ee-1cadab8db18b.png...
processing /content/drive/My Drive/Original/027ab174-e47a-44f8-b85b-372449eeb7d8.png...
processing /content/drive/My Dri

In [ ]:
out_files =  out_dir + '/*.png'
out_names = glob(out_files)


In [ ]:
dice_total=0.0
jacc_total=0.0
image_count=0

In [ ]:
#Calculating dice coefficient

def dice_coef2(a, b):

    hi = a/255 #dividing by 255 so that the values are in the range [0,1] -standardising
    pred = (255-b)/255

    hi= tf.keras.backend.flatten(hi) #flattening the pixel values into a 1D array

    pred= tf.keras.backend.flatten(pred)

    numerator= 2 * tf.keras.backend.sum(hi*pred)

    denominator= tf.keras.backend.sum(hi+pred)
    dice = numerator / denominator

    return dice


In [ ]:
img_names.sort()
img_names1 = img_names[-777:]
out_names.sort()

In [ ]:
# Iterate through the original and segmented image pairs
for gt_img_name, seg_img_name in zip(img_names1, out_names):
    # Ensure the file names match
    if os.path.basename(gt_img_name) == os.path.basename(seg_img_name):
        # Read the original and segmented images in grayscale
        ground_truth = cv2.imread(gt_img_name, 0)
        segmented_img = cv2.imread(seg_img_name, 0)

        # Check if the images were loaded correctly
        if ground_truth is None or segmented_img is None:
            print(f"Error loading images: {gt_img_name} or {seg_img_name}")
            continue

        # Calculate Dice Coefficient for the current pair of images
        dice_score = dice_coef2(ground_truth, segmented_img)
        jacc_score = dice_score / (2-dice_score)
        # Convert TensorFlow tensor to a numerical value (using numpy)
        dice_value = dice_score.numpy()
        jacc_value = jacc_score.numpy()
        # Accumulate the Dice score
        dice_total += dice_value
        jacc_total += jacc_value
        image_count += 1

        # Print the Dice Coefficient for the current image
        print(f"Dice Coefficient for {os.path.basename(gt_img_name)}: {dice_value}")
    else:
        print(f"File mismatch: {os.path.basename(gt_img_name)} and {os.path.basename(seg_img_name)}")

# Calculate and print the average Dice score
if image_count > 0:
    average_dice_score = dice_total / image_count
    average_jacc_score = jacc_total / image_count
    print(f"\nAverage Dice Coefficient for the entire dataset: {average_dice_score}")
    print(f"\nAverage Jaccard Coefficient for the entire dataset: {average_jacc_score}")
else:
    print("No matching images found.")

Dice Coefficient for 5f300fd1-7eec-41ac-bbd1-0f4a943c757c.png: 0.8638102083930536
Dice Coefficient for 5f491efd-a21c-4a80-aaba-70b8876238ac.png: 0.8638670193088712
Dice Coefficient for 5f66d20c-fbfe-44ec-a01b-8a09da24807e.png: 0.8884447739249914
Dice Coefficient for 5f742392-278d-461c-a848-08def2c28878.png: 0.8001318813270547
Dice Coefficient for 5f783f43-0090-4f8a-bd75-9562f4aa2dd5.png: 0.7490073617082891
Dice Coefficient for 5ff7c0c8-d208-4e7d-958d-ca3c2260ff39.png: 0.8476653050480002
Dice Coefficient for 6013bff0-dc4b-4bde-9409-0f6139c3495c.png: 0.8828983156515006
Dice Coefficient for 612bdd3c-da43-4321-a76b-c73c614928d1.png: 0.8714199672687003
Dice Coefficient for 6144887d-a1c1-46db-91a1-f7ff5bf8c8a5.png: 0.8492652532062919
Dice Coefficient for 61575482-91e1-4db9-83c7-687b7ddbbfc4.png: 0.8414245597816807
Dice Coefficient for 61cd8c7d-394d-42d3-9ed4-e09559110bed.png: 0.8730022999899562
Dice Coefficient for 61dfc937-a428-44b4-94fb-d682bdb8dff0.png: 0.887654176569742
Dice Coefficient 

In [ ]:
len(out_names)

776

In [ ]:
len(img_names1)

776

In [ ]:
test_img_path = '/content/drive/My Drive/Original/5f66d20c-fbfe-44ec-a01b-8a09da24807e.png'
test_out_path= '/content/drive/My Drive/Original/test.png'

test_img= cv2.imread(test_img_path, 0)
cv2.imwrite(test_out_path, test_img)



True

In [ ]:
img= cv2.imread(test_out_path, 0)
otsuimg = cv2.imread('/content/drive/My Drive/Otsu_No_Blur/5f66d20c-fbfe-44ec-a01b-8a09da24807e.png', 0)

In [ ]:
print(img.shape)
print(otsuimg.shape)

(1200, 1600)
(1200, 1600)


In [ ]:
d= dice_coef2(img, otsuimg)
ds = d.numpy()
ds

0.19152882986111683